## Linear Regression with Pytorch

In [ ]:
import torch
from torch import nn


class LinearRegression(nn.Module):
    def __init__(self):
        super().__init__()
        self.weights = nn.Parameter(torch.randn(1,requires_grad=True, dtype = torch.float))
        self.bias = nn.Parameter(torch.randn(1,requires_grad=True, dtype = torch.float))
        
    # Forward method to define the computation method in the model
    def forward(self, feature: torch.Tensor) -> torch.Tensor:
        return self.weights * feature + self.bias
        
        